# Aula 4: Como escrever código Escalonável, Flexível e Reutilizável 

Nesta aula, aprenderemos a escrever código que pode ser expandido (escalonável), permite mudanças sem muita dificuldade (flexível) e pode ser utilizado em diferentes contextos (reutilizável). Usaremos como exemplo o desenvolvimento de um pequeno projeto de simulação de dinâmica de partículas, com foco nos desafios de criar um código bem estruturado e eficiente.

Os `notebooks` são ótimos para aprendizado e exploração rápida, mas não são um ambiente ideal para produzir código bem estruturado, uma melhor opção é utilizar scripts, então, todo o código será desenvolvido fora dos `notebooks`, você pode encontrá-lo na pasta [exemplos/particulas](./exemplos/particulas/particles.py).

## Definindo o projeto

Vamos criar uma simulação de $N$ partículas, cuja dinâmica é governada por algumas equações de movimento. Duas equações de movimento serão implementadas

## Mecânica Newtoniana
Neste caso, as partículas são governadas pela famosa segunda lei de Newton

$$
\vec F_i = m_i \vec a_i
$$

em que $\vec F_i, m_i, \vec a_i$ são a força, massa e aceleração da $i$-partículas, respectivamente. Vamos apenas implementar um único tipo força, que confere um núcleo rígido para as partículas,

$$
\vec F_{ij} = \begin{cases} 
F \frac{\vec d_{ij}}{d_{ij}}&,~d_{ij} < r_0 \\
0&,~\text{Caso contrário}
\end{cases}
$$

$\vec F_{ij}$ é força sobre $i$ exercida por $j$, $\vec d_{ij}$ é o vetor deslocamento de $j$ até $i$, $F$ e $r_0$ são parâmetros da força.

## Movimento Browniano
O movimento Browniano modela uma partícula que anda de força completamento aleatória. Suas equações de movimento são

$$
d \vec r_i(t) = D \vec \xi^{(i)}(t) dt
$$

em que $\vec r_i$ é a posição da $i$-ésima partícula e $D$ uma constante relacionada com a difusão. O termo $\vec \xi^{(i)}(t)$ é um ruído branco gaussiano, ou seja, um processo estocástico com as seguintes propriedades

$$
\langle \xi^{(i)}(t) \rangle = 0 \\
\langle \xi^{(i)}_l(t_1) \xi^{(j)}_m(t_2) \rangle = \delta_{ij} \delta_{lm} \delta(t_1 - t_2) \\
$$

O movimento Browniano é fundamental para descrever fenômenos em diversas áreas da ciência, como:

- **Física**: explica o comportamento de partículas suspensas em fluidos, sendo essencial para a teoria cinética dos gases e para o entendimento de processos de difusão.
- **Química**: é importante para modelar reações químicas em soluções, transporte de íons e moléculas em meios líquidos.
- **Biologia**: descreve o movimento de organelas e moléculas dentro de células, além de processos como migração celular e dinâmica populacional.
- **Finanças**: serve de base para modelos matemáticos de variação de preços de ativos, como o movimento aleatório dos preços de ações.
- **Engenharia**: é utilizado em processos de filtragem, controle estocástico e análise de ruídos em sistemas físicos e eletrônicos.

A equação de movimento está na forma diferencial, precisamos obter uma forma finita para usarmos no computador

$$
\vec r_i(t + \Delta t) = \vec r_i(t) +  D R_G(t) \sqrt{\Delta t}
$$

em que $\Delta t$ é o passo temporal e $R_G$ é uma realização de um processo gaussiano normal.

<details>
<summary>Para quem está interessado, clique aqui para ver como chegar na forma finita das equações de movimento</summary>
Primeiro integramos as equações de movimento

$$
\int_{\vec r(0)}^{\vec r(t)} \vec r(t) = \int_0^t D\xi(t) dt \Rightarrow \vec r(r) - \vec r(0) = \int_0^t D \xi(t) dt
$$

Prosseguindo, calcularemos a média e variância de cada componente de $\vec r(t)$. 

$$
\vec \mu = \langle \vec r(t) \rangle = \langle D\int_0^t \xi(t) dt + \vec r(0) \rangle = \int_0^t D\overbrace{\langle \xi(t) \rangle}^0 dt + \vec r(0) = \vec r(0)
$$

como $\vec r(t) = x(t)\hat i + y(t) \hat j$, com $x$ e $y$ possuindo a mesma forma matemática (ambos são descritos por $\int D\xi(t) dt + (x \text{ ou }y)(0)$, apenas com $\xi$ independente entre as coordenadas), calcularemos apenas a variância de $x$, pois o resultado para $y$ é o mesmo. 

$$
\begin{aligned}
\sigma_x^2 &= \big\langle \big[x(t) -  \langle x(t) \rangle \big]^2 \big\rangle = \langle x(t)^2 \rangle - \overbrace{\langle x(t) \rangle^2}^{x(0)^2} = \bigg\langle \bigg[\int_0^t D\xi(t_1)dt_1+x(0) \bigg] \bigg[\int_0^t D\xi(t_2)dt_2 + x(0) \bigg] \bigg\rangle - x(0)^2  = \\
&= \bigg\langle D^2\int_0^t \int_0^t \xi(t_1)\xi(t_2)dt_1dt_2 + 2Dx(0)\int_0^t \xi(t')dt' + x(0)^2 \bigg\rangle - x(0)^2 = \\ 
&= D^2\bigg\langle \int_0^t \int_0^t \xi(t_1)\xi(t_2)dt_1dt_2 \bigg\rangle + 2Dx(0) \overbrace{\bigg\langle\int_0^t \xi(t')dt'\bigg\rangle}^{0} + x(0)^2 - x(0)^2 = \\ 
&= D^2\bigg\langle \int_0^t \int_0^t \xi(t_1)\xi(t_2)dt_1dt_2 \bigg\rangle =  D^2 \int_0^t \int_0^t \overbrace{\langle\xi(t_1)\xi(t_2)\rangle}^{\delta(t_1 - t_2)} dt_1dt_2 = \\
&= D^2 \int_0^t \overbrace{\int_0^t \delta(t_1 - t_2) dt_1}^{1}dt_2 = D^2\int_0^t dt_2 = D^2t
\end{aligned}
$$

se calcularmos os momentos de $\vec r(t)$ de ordem maior, vamos obter sempre 0, loga a distribuição de probabilidades de cada componente de $\vec r(t)$ é um gaussiano com média $\vec r(0)$ e desvio padrão $D \sqrt{t}$ em ambas as coordenadas, então,

$$
r_i(t) \sim \mathcal{N}\big(\mu = r_i(0),~\sigma^2 = D^2 t\big) \Rightarrow p(r_i(t)) = \frac{1}{D\sqrt{2\pi t}} \exp\bigg(-\frac{\big[r_i(t) - r_i(0)\big]^2}{2D^2t} \bigg)
$$

não é difícil verificar que 

$$
r_i(t) ~|~ r_i(t_0) \sim \mathcal{N}\big(\mu = r_i(t_0),~\sigma^2 = D^2 (t-t_0)\big)
$$

logo, se soubermos o valor de $r_i(t)$, temos que uma realização de $r_i(t+\Delta t)$ provém de um gaussiana com média $r_i(t)$ e desvio padrão $D \sqrt{t+\Delta t - t} = D \sqrt{\Delta t}$. Computadores são bons em gerar realização de uma gaussiana normal, então é desejável transformar um gaussiana normal em uma com a média e desvio padrão desejado. Podemos fazer isso assim

$$
G \sim \mathcal{N}\big(\mu=0, \sigma^2 = 1) \Rightarrow G* D\sqrt{\Delta t} + r_i(t) \sim \mathcal{N}\big(\mu=r_i(t), \sigma^2 = D^2 \Delta t)
$$

portanto,

$$
r_i(t+\Delta t) = r_i(t) + D G \sqrt{t}
$$
</details>

## Organização do projeto

- Identificar as diferentes partes que compõem uma simulação de dinâmica de partículas e criar uma classe para cada uma delas, como: 
    - O estado do sistema.
    - Todas as configuração da simulação, como a geometria do espaço, constantes do potencial, etc.

- Tudo que compõem uma simulação será encapsulada em uma classe mãe, chamada de `Simulation`. Dessa forma, podemos completamente separar simulação e visualização, fazendo com que o código da visualização apenas tenhas uma referência de `Simulation`.

- O principal método de `Simulation` é o `step()`, que avança o sistema em um único passo temporal.

## Exercício 1

Utilizando o movimento Browniano, faça uma simulação com as seguintes configurações

- 100 partículas
- Tempo final igual a 100
- $D = 4$
- $\Delta t = 1$

Durante a simulação, guarde a trajetória de todas as partículas, ou seja, as posições em cada passo temporal (Sugestão: armazena as trajetórias em um array com shape `(Número de partículas, Número de passos temporais executados, 2)`). Então, calcule o desvio quadrático médio ($MSD$) da trajetória de cada partícula, faça a média do $MSD$ entre todas as partículas, e, por fim, calcule o coeficiente angular da curva $\langle MSD \rangle \times \Delta t$

> OBS: No caso do movimento Browniano, podemos analiticamente calcular o valor desse coeficiente, o resultado deve ser $2 D^2$

---
### Explicação do MSD
Seja $\vec r(t)$ a posição da partícula em função do tempo, então o MSD é definido da seguinte forma
$$
\text{MSD}(\Delta t) = \langle |\vec r(t+ \Delta t) - \vec r(t)|^2 \rangle_{t}
$$

> OBS: O $\Delta t$ aqui não é o passo temporal da simulação, é simplesmente algum intervalo de tempo
---
